In [ ]:

import boto3
from botocore.config import Config

my_config = Config(
    region_name = 'us-west-2'
)


s3_client = boto3.client('s3',config=my_config)

In [ ]:
bucket_name = 'kfs3-west-dest-dev'
print(bucket_name)

In [ ]:
#s3_client.create_bucket(
#    Bucket=bucket_name
#)

In [ ]:
sns_client = boto3.client('sns',config=my_config)

In [ ]:
import os
topic_account = os.environ['PRODUCER_ACCOUNT_NO']
topic_arn = 'arn:aws:sns:us-west-2:{}:MyTopic'.format(topic_account)
print(topic_arn)

In [ ]:
sns_topic_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "s3.amazonaws.com"  
             },
            "Action": "sns:Publish",
            "Resource": topic_arn,
            "Condition": {
                "ArnLike": {"AWS:SourceArn": f"arn:aws:s3:*:*:{bucket_name}"},
                "StringEquals": { "aws:SourceAccount": topic_account}
            },
        },
    ],
}

In [ ]:
print(sns_topic_policy)

In [ ]:
import json
sns_client.set_topic_attributes(
    TopicArn=topic_arn,
    AttributeName='Policy',
    AttributeValue=json.dumps(sns_topic_policy),
)


In [ ]:
s3_resource = boto3.resource('s3')

In [ ]:
bucket_notification = s3_resource.BucketNotification(bucket_name)

In [ ]:
s3_notification_config = {
    'TopicConfigurations': [
        {
            'TopicArn': topic_arn,
            'Events': [
                's3:ObjectCreated:*',
            ],
        },
    ],
}

In [ ]:
bucket_notification.put(NotificationConfiguration=s3_notification_config)


In [ ]:
key = 'key-' + str(random.randint(1, 9999999))
print(key)

In [ ]:
s3_resource.Object(bucket_name, key).put(Body="here's some shit", StorageClass='REDUCED_REDUNDANCY')

In [ ]:
lo = s3_client.list_objects(
    Bucket=bucket_name
)

In [ ]:
print(lo)

In [ ]:
contents = lo['Contents']

In [ ]:
for c in contents:
    print(c['Key'])
    s3_client.delete_object(
        Bucket=bucket_name,
        Key=c['Key']
    )
    

In [ ]:
s3_client.delete_bucket(
    Bucket=bucket_name
)